In [1]:
import tensorflow as tf
import string
import requests

In [2]:
response= requests.get("https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt")

In [3]:
#response.text

In [4]:
data = response.text.split("\n")

In [5]:
data[0]

'This is the 100th Etext file presented by Project Gutenberg, and'

In [6]:
data = data[253:]
data[0]

'  From fairest creatures we desire increase,'

In [8]:
len(data)

124204

In [9]:
data = " ".join(data)

In [11]:
def clean_text(doc):
    tokens = doc.split()
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word.lower() for word in tokens]

    return tokens

In [12]:
tokens = clean_text(data)

print(tokens[:50])

['from', 'fairest', 'creatures', 'we', 'desire', 'increase', 'that', 'thereby', 'beautys', 'rose', 'might', 'never', 'die', 'but', 'as', 'the', 'riper', 'should', 'by', 'time', 'decease', 'his', 'tender', 'heir', 'might', 'bear', 'his', 'memory', 'but', 'thou', 'contracted', 'to', 'thine', 'own', 'bright', 'eyes', 'feedst', 'thy', 'lights', 'flame', 'with', 'selfsubstantial', 'fuel', 'making', 'a', 'famine', 'where', 'abundance', 'lies', 'thy']


In [13]:
len(tokens)

898199

In [14]:
len(set(tokens))

27956

In [35]:
length = 50 + 1     ## 50 tane başta benim eğittiğim +1 ise predict(tahmin)
lines = []

In [36]:
for i in range(length, len(tokens)):
    seq = tokens[i - length: i]
    line = " ".join(seq)
    lines.append(line)
    if i > 200000:
        break
print(len(lines)) 

199951


In [37]:
lines[1]

'fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self thy'

In [38]:
tokens[50]

'self'

## Build LSTM Model and Prepare X and Y

In [19]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [39]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [21]:
sequences = np.array(sequences)

In [22]:
x, y = sequences[:, :-1], sequences[:, -1]

In [23]:
x[0]

array([   47,  1408,  1264,    37,   451,  1406,     9,  2766,  1158,
        1213,   171,   132,   269,    20,    24,     1,  4782,    87,
          30,    98,  4781,    18,   715,  1263,   171,   211,    18,
         829,    20,    27,  3807,     4,   214,   121,  1212,   153,
       13004,    31,  2765,  1847,    16, 13003, 13002,   754,     7,
        3806,    99,  2430,   466,    31])

In [24]:
y[0]

307

In [25]:
vocab_size = len(tokenizer.word_index) + 1

In [26]:
y = to_categorical(y, num_classes = vocab_size)

In [27]:
seq_length = x.shape[1]

## LSTM Model

In [28]:
model = Sequential()

model.add(Embedding(vocab_size, 50, input_length = seq_length))
model.add(LSTM(100, return_sequences = True))
model.add(LSTM(100))
model.add(Dense(100, activation = "relu"))
model.add(Dense(vocab_size, activation = "softmax"))

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            650450    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 13009)             1313909   
Total params: 2,115,259
Trainable params: 2,115,259
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [31]:
model.fit(x, y, batch_size = 256, epochs = 3)

Epoch 1/3
782/782 [==============================] - 566s 723ms/step - loss: 6.8804 - accuracy: 0.0312
Epoch 2/3
782/782 [==============================] - 576s 737ms/step - loss: 6.5224 - accuracy: 0.0457
Epoch 3/3
782/782 [==============================] - 576s 736ms/step - loss: 6.2998 - accuracy: 0.0630


In [50]:
seed_text = lines[12343]

In [54]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
    text = []
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating = "pre")

        y_predict = model.predict_classes(encoded)
        predict_word = ""

        for word, index in tokenizer.word_index.items():
            if index == y_predict:
                predict_word = word
                break
        
        seed_text = seed_text + " " + predict_word
        text.append(predict_word)

    return (" ".join(text))

In [56]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 10)

'not i have not not i have not not i'

In [57]:
seed_text

'home of love if i have ranged like him that travels i return again just to the time not with the time exchanged so that my self bring water for my stain never believe though in my nature reigned all frailties that besiege all kinds of blood that it could so'